In [57]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

class SpeedDataset(Dataset):
    def __init__(self, directory, sequence_length=5):
        self.data = []
        self.sequence_length = sequence_length
        self.max_speed = 0  # Variable to store the maximum speed found
        self.preprocess_data(directory)
        # Calculate number of classes based on the range from 30 to 105
        self.num_classes = (105 - 30) // 10 + 1  # From class 30-39, ..., 100-105

    def preprocess_data(self, directory):
        for filename in os.listdir(directory):
            if filename.endswith('.txt'):
                speed = float(filename.split('_')[-1].replace('.txt', ''))
                # Update maximum speed if current speed is greater
                if speed > self.max_speed:
                    self.max_speed = speed
                # Convert speed to class, starting range from 30
                if speed < 30:
                    speed_class = 0  # Handle speeds below 30 as class 0
                else:
                    speed_class = (int(speed) - 30) // 10
                filepath = os.path.join(directory, filename)
                with open(filepath, 'r') as file:
                    track_data = {}
                    for line in file:
                        frame, track_id, x1, y1, x2, y2 = map(float, line.strip().split(',')[:6])
                        if track_id not in track_data:
                            track_data[track_id] = []
                        track_data[track_id].append([x1,y1,x2,y2,(x1+x2)/2, (y1+y2)/2])

                    for track_id, frames in track_data.items():
                        if len(frames) >= self.sequence_length:
                            features = []
                            for i in range(1, len(frames)):
                                current_frame = frames[i]
                                previous_frame = frames[i-1]
                                data_d = []
                                for i in range(6):
                                    data_d.append(current_frame[i])
                                for j in range(4,6):
                                    data_d.append(current_frame[j] - previous_frame[j])
                                features.append(data_d)
                                
                            for i in range(len(features) - self.sequence_length + 1):
                                sequence = features[i:i + self.sequence_length]
                                self.data.append((sequence, speed_class))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs, output = self.data[idx]
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(output, dtype=torch.long)


In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        # Move the positional encoding to the same device as the input tensor
        encoding = self.encoding[:, :x.size(1)].to(x.device)
        return x + encoding.detach()


class SpeedPredictor(nn.Module):
    def __init__(self, sequence_length, feature_size, embedding_dim, hidden_dim, num_classes, n_heads=2, n_layers=2):
        super(SpeedPredictor, self).__init__()
        self.conv1 = nn.Conv1d(feature_size, embedding_dim, kernel_size=1)
        self.fc2 = nn.Linear(embedding_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim,num_classes)
        self.relu = nn.ReLU()
        self.pos_encoder = PositionalEncoding(embedding_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=n_heads,
            batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(
            self.encoder_layer,
            num_layers=n_layers)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1(x)
        x = x.permute(0, 2, 1)
        x = self.pos_encoder(x)  # Apply positional encoding
        x = self.transformer_encoder(x)
        x = x[:, -1, :]  # Get the output from the last timestep
        x = self.fc2(x)
        return self.relu(self.fc1(x))



In [59]:
import torch

# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [60]:
def train(model, train_loader, test_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct_train = 0
        total_train = 0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            correct_train += (predicted == targets).sum().item()
            total_train += targets.size(0)

        train_accuracy = 100 * correct_train / total_train

        if epoch % 10 == 0 or epoch == epochs - 1:
            model.eval()
            correct_test = 0
            total_test = 0
            all_preds = []
            all_targets = []
            with torch.no_grad():
                for inputs, targets in test_loader:
                    inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs.data, 1)
                    correct_test += (predicted == targets).sum().item()
                    total_test += targets.size(0)
                    all_preds.extend(predicted.cpu().numpy())
                    all_targets.extend(targets.cpu().numpy())
            test_accuracy = 100 * correct_test / total_test
            print(f'Epoch {epoch+1}: Train Loss: {total_loss / len(train_loader)}, '
                  f'Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%')



In [61]:
train_dataset = SpeedDataset('outputs', sequence_length=5)
test_dataset = SpeedDataset('test', sequence_length=5)
max_classes = max(train_dataset.num_classes, test_dataset.num_classes)
model = SpeedPredictor(sequence_length=10, feature_size=8, embedding_dim=64, hidden_dim=32, num_classes=max_classes,n_heads =2 , n_layers = 2)
model.to(device)  # Move model to the appropriate device
criterion = nn.CrossEntropyLoss().to(device)  # Move criterion to the appropriate device
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)
# Train and Evaluate
train(model, train_loader, test_loader, criterion, optimizer, epochs=500)


Epoch 1: Train Loss: 2.079562432584707, Train Accuracy: 18.36%, Test Accuracy: 18.88%
Epoch 11: Train Loss: 2.0794413089752197, Train Accuracy: 18.44%, Test Accuracy: 18.88%


KeyboardInterrupt: 

In [ ]:
def print_confusion_matrix_and_report(all_targets, all_preds):
    print(confusion_matrix(all_targets, all_preds))
    print(classification_report(all_targets, all_preds, target_names=[f'Class {30 + i * 10}-{39 + i * 10}' for i in range(max(all_targets) + 1)]))


In [ ]:
def evaluate(model, test_loader):
    model.eval()
    correct_test = 0
    total_test = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            correct_test += (predicted == targets).sum().item()
            total_test += targets.size(0)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    test_accuracy = 100 * correct_test / total_test
    print("Confusion Matrix and Classification Report:")
    print_confusion_matrix_and_report(all_targets, all_preds)
    return test_accuracy
evaluate(model, test_loader)  